In [ ]:
from torchcrf import CRF
import torch
import spacy
import torch.nn as nn
from torch.optim import AdamW
from transformers import get_cosine_schedule_with_warmup
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
from metrics import f1score
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from multiprocessing import cpu_count
from platform import system
from os import environ
from sklearn.metrics import classification_report
from pandas import read_csv
from ast import literal_eval
import numpy as np
from itertools import chain

environ["TOKENIZERS_PARALLELISM"] = "false"
nlp = spacy.load("en_core_web_sm")
pl.seed_everything(seed=42)

In [ ]:
convert_to_one_hot = lambda values: np.eye(len(tag2idx), dtype=int)[values].tolist()

strip_X = lambda values: [[v for v in values_ if v != tag2idx['X']] for values_ in values]

In [ ]:
def post_pad_sequences(sequences, max_len=None, pad_value=0):
    longest = max([len(x) for x in sequences])
    if max_len is not None and max_len > longest:
        print("Unnecessary extra padding detected!\nSequences will be only be padded to the longest length")
    sequences = [(x + [pad_value]*(longest-len(x))) for x in sequences]
    return sequences if max_len is None or max_len >= longest else [x[:max_len] for x in sequences]

In [ ]:
LEARNING_RATE = 1.5e-1
BATCH_SIZE = 128
EPOCHS = 25
MAX_LEN = None
N_JOBS = cpu_count() if system() != "Windows" else 0

tag2idx = {'B': 0, 'I': 1, 'O': 2, 'E': 3, 'S': 4, 'X': 5}
pos2idx = {"NOUN": 0, "VERB": 1, "ADJ": 2, "ADV": 3, "PRON": 4, "OTHER": 5}

In [ ]:
class CRF4NER(pl.LightningModule):
    def __init__(self, 
                 num_tags=len(tag2idx),
                 warmup_steps=2,
                 total_steps=1024,
                 train_dataset=None,
                 val_dataset=None,
                 test_dataset=None):

        super().__init__()
        self.crf = CRF(num_tags=num_tags, batch_first=True)
        ## Hyperparameters ##
        self.batch_size = BATCH_SIZE
        self.learning_rate = LEARNING_RATE
        self.warmup_steps = warmup_steps
        self.total_steps = total_steps
        ## Datasets ##
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.test_dataset = test_dataset

    
    def train_dataloader(self):
        return DataLoader(self.train_dataset, 
                          batch_size=self.batch_size,
                          shuffle=True,
                          num_workers=N_JOBS,
                          drop_last=False)


    def val_dataloader(self):
        return DataLoader(self.val_dataset, 
                          batch_size=self.batch_size,
                          num_workers=N_JOBS,
                          drop_last=False)


    def test_dataloader(self):
        return DataLoader(self.test_dataset, 
                          batch_size=self.batch_size,
                          num_workers=N_JOBS,
                          drop_last=False)

    
    def predict_dataloader(self):
        return DataLoader(self.test_dataset, 
                          batch_size=self.batch_size,
                          num_workers=N_JOBS,
                          drop_last=False)
        
        
    def forward(self, input_ids):
        pass


    def _shared_evaluation_step(self, batch, batch_idx):
        ids, masks, lbls = batch
        loss = -self.crf(ids, lbls, mask=masks)
        pred = self.crf.decode(ids, mask=masks)
        r, p, f1 = f1score(lbls.tolist(), pred)
        return loss, r, p, f1
    
        
    def training_step(self, batch, batch_idx):
        loss, r, p, f1 = self._shared_evaluation_step(batch, batch_idx)
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_recall", r, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_precision", p, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_f1score", f1, on_step=False, on_epoch=True, prog_bar=True)
        return loss


    def validation_step(self, batch, batch_idx):
        loss, r, p, f1 = self._shared_evaluation_step(batch, batch_idx)
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val_recall", r, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val_precision", p, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val_f1score", f1, on_step=False, on_epoch=True, prog_bar=True)

    
    def test_step(self, batch, batch_idx):
        loss, r, p, f1 = self._shared_evaluation_step(batch, batch_idx)
        self.log("test_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_recall", r, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_precision", p, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_f1score", f1, on_step=False, on_epoch=True, prog_bar=True)


    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        ids, masks, _ = batch 
        return self.crf.decode(ids, mask=masks)


    def configure_optimizers(self):
        return AdamW(self.parameters(), 
                     amsgrad=True, 
                     lr=self.learning_rate)

        scheduler = get_cosine_schedule_with_warmup(optimizer=optimizer,
                                                    num_warmup_steps=self.warmup_steps,
                                                    num_training_steps=self.total_steps)

        lr_scheduler = {
            'scheduler': scheduler, 
            'interval': 'epoch', 
            'frequency': 1
        }
        
        return [optimizer], [lr_scheduler]

In [ ]:
data = read_csv("../data/train_290818.txt", 
                sep=" ",
                header=None, 
                encoding="utf-8").values.tolist()

text = [literal_eval(words) for (words, _, _) in data]
text = [[token.pos_ for token in nlp(' '.join(s))] for s in text]
text = [[p if (p == "NOUN" or p == "ADV" or p == "VERB" or p == "ADJ" or p == "PRON") else "OTHER" for p in sent] for sent in text]
labels = [[l.split('-')[0] for l in literal_eval(labels)] for (_, labels, _) in data]

primitive_lens = [len(x) for x in text]
text = post_pad_sequences(text, max_len=MAX_LEN, pad_value="OTHER")
fl = len(text[0])

encoded_input = [convert_to_one_hot([pos2idx[p] for p in sent]) for sent in text]
attn_masks = [([1]*pl + [0]*(fl - pl))[:MAX_LEN] for pl in primitive_lens]

labels = [(l + ['X']*(fl - len(l)))[:MAX_LEN] for l in labels]
extended_labels = [[tag2idx[l] for l in lbls] for lbls in labels]

L = len(extended_labels)

dataset = TensorDataset(torch.LongTensor(encoded_input),
                        torch.BoolTensor(attn_masks),
                        torch.LongTensor(extended_labels))

train_sz, val_sz = L-int(0.1*L), int(0.1*L)
train_dataset, val_dataset = random_split(dataset, (train_sz, val_sz))                                                                       

In [ ]:
data = read_csv("../data/test_290818.txt", 
                sep=" ",
                header=None, 
                encoding="utf-8").values.tolist()

text = [literal_eval(words) for (words, _, _) in data]
text = [[token.pos_ for token in nlp(' '.join(s))] for s in text]
text = [[p if (p == "NOUN" or p == "ADV" or p == "VERB" or p == "ADJ" or p == "PRON") else "OTHER" for p in sent] for sent in text]
labels = [[l.split('-')[0] for l in literal_eval(labels)] for (_, labels, _) in data]

primitive_lens = [len(x) for x in text]
text = post_pad_sequences(text, max_len=MAX_LEN, pad_value="OTHER")
fl = len(text[0])

encoded_input = [convert_to_one_hot([pos2idx[p] for p in sent]) for sent in text]
attn_masks = [([1]*pl + [0]*(fl - pl))[:MAX_LEN] for pl in primitive_lens]
labels = [(l + ['X']*(fl - len(l)))[:MAX_LEN] for l in labels]
extended_labels = [[tag2idx[l] for l in lbls] for lbls in labels]

test_dataset = TensorDataset(torch.LongTensor(encoded_input),
                             torch.BoolTensor(attn_masks),
                             torch.LongTensor(extended_labels))                                                                                                                    

In [ ]:
WARMUP_RATIO = 0.01
TOTAL_STEPS = len(train_dataset) // BATCH_SIZE
WARMUP_STEPS = int(WARMUP_RATIO * TOTAL_STEPS)

In [ ]:
model = CRF4NER(train_dataset=train_dataset,
                val_dataset=val_dataset,
                test_dataset=test_dataset,
                warmup_steps=WARMUP_STEPS,
                total_steps=TOTAL_STEPS)

earlystopping_callback = EarlyStopping(monitor="val_f1score", 
                                       min_delta=1e-4, 
                                       patience=EPOCHS, 
                                       mode="max")

checkpoint_callback = ModelCheckpoint(dirpath="./",
                                      filename="crf-ner-val-f1score",
                                      save_top_k=1, 
                                      mode="max",
                                      monitor="val_f1score",
                                      save_weights_only=True)

trainer = pl.Trainer(accelerator="gpu",
                     max_epochs=EPOCHS,
                     precision=16,
                     log_every_n_steps=1,
                     callbacks=[earlystopping_callback, checkpoint_callback])

In [ ]:
trainer.fit(model)

In [ ]:
model.load_state_dict(torch.load(f"./crf-ner-val-f1score.ckpt")["state_dict"])
trainer.test(model)

In [ ]:
preds = list(chain.from_iterable(trainer.predict(model)))

preds = np.array(post_pad_sequences(preds, pad_value=tag2idx['X'])).flatten()
lbls = np.array(extended_labels).flatten()

clf_rp = classification_report(lbls,
                               preds,
                               zero_division=0,
                               labels=[0, 1, 2, 3, 4, 5],
                               target_names=['B', 'I', 'O', 'E', 'S', 'X'])

print("-"*80)
print(clf_rp)
print("-"*80)